## Libraries & global variables

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add path to custom modules
import sys
import os
os.environ['PATH'] = os.environ['PATH'] + ':/Library/TeX/texbin'
this_path = os.path.abspath('') 
parent_dir = os.path.dirname(this_path)  
sys.path.append(parent_dir)

# Import custom modules
from Modules import Parameters, CustomFunctions as CF

### Colors

In [ ]:
# color scheme
color_control = CF.lighten_color(Parameters.green, 0.8)
color_control1 = Parameters.green
color_immunecells = Parameters.orange
color_cytokines = Parameters.pink #red_purp
color_bloodtests1 = Parameters.lavander
color_bloodtests2 = Parameters.violet
color_scores = Parameters.light_red_purp #coral
color_demographics = Parameters.light_blue #pine_green
color_CT = Parameters.light_blue
color_0 = Parameters.dark_blue
color_0_light = Parameters.light_blue
color_1 = Parameters.red_purp
color_1_light = Parameters.light_red_purp
color_background = CF.lighten_color(Parameters.grey_2, 0.6)
color_IC_multiv = Parameters.blue_green
color_CK_multiv = Parameters.coral
color_BM_multiv = Parameters.queen_blue
color_ICB_multiv = Parameters.eton_blue

### Parameters

In [ ]:
## Multivariate sets
allinput_set = Parameters.allinput_set
immunecells_set = Parameters.immunecells_set
demographics_set = Parameters.demographics_set
cytokines_set = Parameters.cytokines_set 
biomarkers_set = Parameters.biomarkers_set
FC_set = Parameters.FC_set
Dem_set = Parameters.Dem_set
CK_set = Parameters.CK_set
BM_set =  Parameters.BM_set
scores_set = Parameters.scores_set

## Minimum NPV
min_NPV = Parameters.min_NPV

## Paths
path_results = Parameters.path_results
path_figures = Parameters.path_figures
exp_description = Parameters.exp_description
exp_multiv_description = Parameters.exp_multiv_description
foldername_univ = Parameters.foldername_univ
foldername_multiv = Parameters.foldername_multiv

## Univariate models

In [ ]:
# Parameters
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
capsize = 0
width = 0.7

# Variables
len_dem = len(Dem_set)
len_fc = len(FC_set)
len_ck = len(CK_set)
len_bm = len(BM_set)
len_scores = len(scores_set)
len_cytok = len(cytokines_set)
len_immcells = len(immunecells_set)

### Import files

In [ ]:
foldername_univ = 'UnivModels_DOnsetMin#0#Max#30_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NanMask#True/'
foldername_univ0 = 'UnivModels_DOnsetMin#0#Max#30_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NanMask#True_Std#False/'

In [ ]:
path_import = path_results+foldername_univ

filename = 'performances_test.xlsx'
df_performances = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)

path_import = path_results+foldername_univ0

filename_0 = 'performances_minNPV_test.xlsx'
df_performances_0 = pd.read_excel(path_import+filename_0, engine='openpyxl', header=[0, 1], index_col=0)

In [ ]:
# Clean nans
df_performances = df_performances.where(df_performances.notnull().values, 0)
df_performances_0 = df_performances_0.where(df_performances_0.notnull().values, 0)

### Scores

In [ ]:
Data = df_performances
Data_0 = df_performances_0
scores = [element[0] for element in Data.columns]
scores = list(dict.fromkeys(scores))
scores_0 = [element[0] for element in Data_0.columns]
scores_0 = list(dict.fromkeys(scores_0))

index = [col.replace('LR: ', '') for col in Data.index]
Data.index = index
index = [col.replace('LR: ', '') for col in Data_0.index]
Data_0.index = index

### Figures

#### AUC sorting

In [ ]:
Data_l = Data.copy()
features = allinput_set

In [ ]:
# Labels
old_labels = [col for col in Data_l.index]
new_labels = CF.change_names(old_labels)
x_labels = np.array(new_labels) # or list(X_data.columns.values)

# Sorting
sorting_score = 'roc_auc_score'
y = Data_l[(sorting_score, 'Mean')].values
idx_sorted = []
for set_col in (Dem_set, immunecells_set, cytokines_set, BM_set, scores_set):
    sub_idx = np.array([i for i in range(len(old_labels)) if old_labels[i] in set_col])
    min_sub_idx = min(sub_idx)
    sub_idx_sorted = min_sub_idx + np.argsort(y[sub_idx])
    idx_sorted.extend(sub_idx_sorted)
idx_sorted = np.array(idx_sorted)
x_labels_sorted = np.array(new_labels)[idx_sorted]
old_labels_sorted =  np.array(old_labels)[idx_sorted]


# Colors
colors = [color_demographics]*len_dem + [color_immunecells]*len_immcells
colors += [color_cytokines]*len_cytok + [color_bloodtests1]*len_bm + [color_scores]*len_scores

#### AUC - Plot with text

In [ ]:
x_range = np.arange(len(features), dtype=float)
dx = 0.2
x_positions_labels = x_range
x_positions_labels[len_dem:] += dx
x_positions_labels[len_dem+len_immcells:] += dx
x_positions_labels[len_dem+len_immcells+len_cytok:] += dx
x_positions_labels[len_dem+len_immcells+len_cytok+len_bm:] += dx
x_positions_bars = x_positions_labels

magnification = 1.5
ratio = 2./5.25

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'roc_auc_score'

y = Data.loc[old_labels_sorted, (score, 'Mean')].values
y_lower = Data.loc[old_labels_sorted, (score, 'CI_lower')].values
y_upper = Data.loc[old_labels_sorted, (score, 'CI_upper')].values

ax.bar(x_positions_bars, y, color=colors, alpha=0.8, width=width, lw=lines_w)
ax.errorbar(x_positions_bars, y, yerr=[y-y_lower,y_upper-y], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('auc')
ax.set_ylim([0.36, .84])
ax.set_yticks([0.4, 0.5, 0.6, 0.7, 0.8])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    
    
max_score = max(y)
idx1 = len(scores_set)
max_prog_score = max(y[-idx1:])

idx = np.argmax(y)
ax.axhline(y=max_score, ls='--', color=colors[idx], alpha=0.4, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted[idx], max_score)
ax.text(1.025, 1.1*max_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)
idx = len(y)-idx1+np.argmax(y[-idx1:])
ax.axhline(y=max_prog_score, ls='--', color=colors[idx], alpha=1, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted[idx], max_prog_score)
ax.text(1.025, 1.*max_prog_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

if score=='roc_auc_score':
    name_score = 'auc'

filename = '%sUniv' % (name_score) + '.pdf'    
saving_str = path_figures + filename
#plt.savefig(saving_str, bbox_inches='tight') # dpi=1000, 
plt.show()

#### AUC - Plot with no text

In [ ]:
magnification = 1.35
ratio = 2./5.
CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'roc_auc_score'

y = Data.loc[old_labels_sorted, (score, 'Mean')].values
y_lower = Data.loc[old_labels_sorted, (score, 'CI_lower')].values
y_upper = Data.loc[old_labels_sorted, (score, 'CI_upper')].values

ax.bar(x_positions_bars, y, color=colors, alpha=0.8, width=width, lw=lines_w)
ax.errorbar(x_positions_bars, y, yerr=[y-y_lower,y_upper-y], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('auc')
ax.set_ylim([0.36, .84])
ax.set_yticks([0.4, 0.5, 0.6, 0.7, 0.8])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    
    
max_score = max(y)
idx1 = len_scores
max_prog_score = max(y[-idx1:])

idx = np.argmax(y)
ax.axhline(y=max_score, ls='--', color=colors[idx], alpha=0.4, lw=0.5)
idx = len(y)-idx1+np.argmax(y[-idx1:])
ax.axhline(y=max_prog_score, ls='--', color=colors[idx], alpha=1, lw=0.5)

if score=='roc_auc_score':
    name_score = 'auc'

filename = '%sUniv_noText' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str, bbox_inches='tight') # dpi=1000, 
plt.show()

#### NPV (minNPV) - Plot with text

In [ ]:
Data_l = Data_0.loc[features, :].copy()

In [ ]:
magnification = 1.5
ratio = 2./5.25

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'NPV'
name_score = 'npv'

y = Data_l.loc[old_labels_sorted, (score, 'Mean')].values
y_lower = Data_l.loc[old_labels_sorted, (score, 'CI_lower')].values
y_upper = Data_l.loc[old_labels_sorted, (score, 'CI_upper')].values

minNPV_train = min_NPV
delta_minNPV_test = 0.02
mask_npv_high = y_upper>minNPV_train-delta_minNPV_test
x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_low = mask_npv_high==False
x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.4
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=width, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_none = (y>0.)==False
x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.06*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel(name_score)
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    

ax.axhline(y=minNPV_train-delta_minNPV_test, ls='--', color='black', alpha=1, lw=0.5)
my_string = 'min. val.\naccepted:\n%.2f' % (minNPV_train-delta_minNPV_test)
ax.text(1.025, 1.1*max_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'light: npv$<$%.2f' % (minNPV_train-delta_minNPV_test)
ax.text(0.04, 1.1, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'dark: npv$\ge$%.2f' % (minNPV_train-delta_minNPV_test)
ax.text(0.5, 1.1, my_string,
           horizontalalignment='center',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'x: missing'
ax.text(.96, 1.1, my_string,
           horizontalalignment='right',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

#ax.set_yscale('log')
ax.set_ylim([0., 1.])
ax.set_yticks([0., 0.5, 1.])
filename = 'minNPV#%sUniv' % (name_score) + '.pdf'    
saving_str = path_figures + filename
#plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()

#### NPV (minNPV) - Plot with no text

In [ ]:
magnification = 1.35
ratio = 2./5.

CF.SetPlotParams(magnification=magnification, ratio=ratio*0.95, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'NPV'
name_score = 'npv'

y = Data_l.loc[old_labels_sorted, (score, 'Mean')].values
y_lower = Data_l.loc[old_labels_sorted, (score, 'CI_lower')].values
y_upper = Data_l.loc[old_labels_sorted, (score, 'CI_upper')].values

x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_low = mask_npv_high==False
x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.4
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=width, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_none = (y>0.)==False
x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.06*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel(name_score)
ax.set_ylim([0., 1.])
ax.set_yticks([0., 0.5, 1.])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    

minNPV_train = min_NPV
ax.axhline(y=minNPV_train-0.02, ls='--', dashes=(3., 3.), color='black', alpha=1., lw=0.5)


filename = 'minNPV#%sUniv_noText' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()

#### Specificity (minNPV) - Plot with text

In [ ]:
Data_l = Data_0.loc[features, :].copy()

In [ ]:
magnification = 1.5
ratio = 2./5.25

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'specificity'
name_score = 'specificity'

y = Data_l.loc[old_labels_sorted, (score, 'Mean')].values
y_lower = Data_l.loc[old_labels_sorted, (score, 'CI_lower')].values
y_upper = Data_l.loc[old_labels_sorted, (score, 'CI_upper')].values

x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=width, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.4
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.03*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('f. low risk')
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    

y_l = y
y_l[mask_npv_low] = 0
y_l[mask_npv_none] = 0
max_score = max(y_l)
idx1 = len_scores
max_prog_score = max(y_l[-idx1:])

idx = np.argmax(y_l)
ax.axhline(y=max_score, ls='--', dashes=(3., 3.), color=colors[idx], alpha=1, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted[idx], max_score)
ax.text(1.025, 1.8*max_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)
idx = len(y_l)-idx1+np.argmax(y_l[-idx1:])
ax.axhline(y=max_prog_score, ls='--', color=colors[idx], alpha=1, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted[idx], max_prog_score)
ax.text(1.025, 1.4*max_prog_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

ax.set_ylim([0., .6])
ax.set_yticks([0., 0.2, .4, 0.6])
filename = 'minNPV#%sUniv' % (name_score) + '.pdf'    
saving_str = path_figures + filename
#plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()

#### Specificity (minNPV) - Plot with no text

In [ ]:
magnification = 1.35
ratio = 2./5.

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'specificity'
name_score = 'specificity'

y = Data_l.loc[old_labels_sorted, (score, 'Mean')].values
y_lower = Data_l.loc[old_labels_sorted, (score, 'CI_lower')].values
y_upper = Data_l.loc[old_labels_sorted, (score, 'CI_upper')].values

x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=width, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.4
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.03*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('f. low risk')
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    

y_l = y
y_l[mask_npv_low] = 0
y_l[mask_npv_none] = 0
max_score = max(y_l)
idx1 = len_scores
max_prog_score = max(y_l[-idx1:])

idx = np.argmax(y_l)
ax.axhline(y=max_score, ls='--', dashes=(3., 3.), color=colors[idx], alpha=1, lw=0.5)
idx = len(y_l)-idx1+np.argmax(y_l[-idx1:])
ax.axhline(y=max_prog_score, ls='--', color=colors[idx], alpha=1, lw=0.5)

ax.set_ylim([0., 0.6])
ax.set_yticks([0., 0.2, .4, 0.6])
filename = 'minNPV#%sUniv_noText' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()

# Multivariate models

In [ ]:
# Parameters
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
capsize = 0
width = 0.7

# Variables
len_dem = len(Dem_set)
len_fc = len(FC_set)
len_ck = len(CK_set)
len_bm = len(BM_set)
len_scores = len(scores_set)
len_cytok = len(cytokines_set)
len_immcells = len(immunecells_set)

# Models names
multiv_models = ['FC',
                 'FC+Dem',
                 'CK', 
                 'CK+Dem',
                 'BM', 
                 'BM+Dem']
v = [element.replace('LR: ', '').replace('+', ',').replace('.', '') for element in multiv_models]
multiv_models_names = v

### Import files

In [ ]:
foldername_univ = 'UnivModels_DOnsetMin#0#Max#30_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NanMask#True/'
foldername_univ0 = 'UnivModels_DOnsetMin#0#Max#30_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NanMask#True_Std#False/'

In [ ]:
path_import = path_results+foldername_univ

filename = 'performances_test.xlsx'
df_performances_univ = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)

path_import = path_results+foldername_univ0

filename_0 = 'performances_minNPV_test.xlsx'
df_performances_univ_0 = pd.read_excel(path_import+filename_0, engine='openpyxl', header=[0, 1], index_col=0)

In [ ]:
# Clean nans
df_performances_univ = df_performances_univ.where(df_performances_univ.notnull().values, 0)
df_performances_univ_0 = df_performances_univ_0.where(df_performances_univ_0.notnull().values, 0)

In [ ]:
foldername_multiv = 'MultivModels_DOnsetMin#0#Max#30_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NansRowGroupwise#50_PCAPercVarTh#5/'

In [ ]:
path_import = path_results+foldername_multiv

filename = 'performances_test.xlsx'
df_performances = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)

filename_0 = 'performances_minNPV_test.xlsx'
df_performances_0 = pd.read_excel(path_import+filename_0, engine='openpyxl', header=[0, 1], index_col=0)

In [ ]:
# Clean nans
df_performances = df_performances.where(df_performances.notnull().values, 0)
df_performances_0 = df_performances_0.where(df_performances_0.notnull().values, 0)

### Scores

In [ ]:
Data_univ = df_performances_univ
Data_univ_0 = df_performances_univ_0
Data = df_performances
Data_0 = df_performances_0
scores = [element[0] for element in Data.columns]
scores = list(dict.fromkeys(scores))
scores_0 = [element[0] for element in Data_0.columns]
scores_0 = list(dict.fromkeys(scores_0))

index = [col.replace('LR: ', '') for col in Data_univ.index]
Data_univ.index = index
index = [col.replace('LR: ', '') for col in Data_univ_0.index]
Data_univ_0.index = index
index = [col.replace('LR: ', '') for col in Data.index]
Data.index = index
index = [col.replace('LR: ', '') for col in Data_0.index]
Data_0.index = index

In [ ]:
features = Dem_set + FC_set + CK_set + BM_set + scores_set
Data_l = Data_univ.loc[features, :].copy()

# Labels
old_labels = [col for col in Data_l.index]
new_labels = CF.change_names(old_labels)
x_labels = np.array(new_labels) # or list(X_data.columns.values)

# Sorting
sorting_score = 'roc_auc_score'
y = Data_l[(sorting_score, 'Mean')].values
idx_sorted = []
for set_col in (Dem_set, FC_set, CK_set, BM_set, scores_set):
    sub_idx = np.array([i for i in range(len(old_labels)) if old_labels[i] in set_col])
    min_sub_idx = min(sub_idx)
    sub_idx_sorted = min_sub_idx + np.argsort(y[sub_idx])
    idx_sorted.extend(sub_idx_sorted)
idx_sorted = np.array(idx_sorted)
x_labels_sorted = np.array(new_labels)[idx_sorted]
old_labels_sorted =  np.array(old_labels)[idx_sorted]


# Colors
colors = [color_demographics]*len_dem
colors += [color_immunecells]*len_fc
colors += [color_cytokines]*len_ck
colors += [color_bloodtests1]*len_bm
colors += [color_scores]*len_scores

Data_combo = pd.concat([Data_univ.loc[old_labels_sorted, :], Data.loc[multiv_models, :]], axis=0)

colors += [color_IC_multiv]*2 # FC models
colors += [color_CK_multiv]*2 # CK models
colors += [color_BM_multiv]*2 # BM models
old_labels_sorted_combo = list(old_labels_sorted) + multiv_models
x_labels_sorted_combo = list(x_labels_sorted) + v 

### Plot

#### AUC - Plot with text

In [ ]:
x_range = np.arange(len(x_labels_sorted_combo), dtype=float)
dx = 0.2
x_positions_labels = x_range
x_positions_labels[len_dem:] += dx
x_positions_labels[len_dem+len_fc:] += dx
x_positions_labels[len_dem+len_fc+len_ck:] += dx
x_positions_labels[len_dem+len_fc+len_ck+len_dem:] += dx
x_positions_labels[len_dem+len_fc+len_ck+len_dem+len_scores:] += dx
x_positions_labels[len_dem+len_fc+len_ck+len_dem+len_scores+2:] += dx
x_positions_labels[len_dem+len_fc+len_ck+len_dem+len_scores+4:] += dx
x_positions_bars = x_positions_labels

magnification = 1.35
ratio = 2./5.

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'roc_auc_score'

Data_to_use = Data_combo

y = Data_to_use.loc[old_labels_sorted_combo, (score, 'Mean')].values
y_lower = Data_to_use.loc[old_labels_sorted_combo, (score, 'CI_lower')].values
y_upper = Data_to_use.loc[old_labels_sorted_combo, (score, 'CI_upper')].values

ax.bar(x_positions_bars, y, color=colors, alpha=0.8, width=width, lw=lines_w)
ax.errorbar(x_positions_bars, y, yerr=[y-y_lower,y_upper-y], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_combo, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('auc')
ax.set_ylim([0.3, .9])
ax.set_yticks([0.3, 0.6, 0.9])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    
    
max_score = max(y)
idx1 = len_dem + len_fc + len_ck + len_bm
idx2 = idx1 + len_scores
idx3 = idx2 + 1
max_score_progscores = max(y[idx1:idx2])
IC_Dem = y[idx3]

idx = np.argmax(y)
ax.axhline(y=max_score, ls='--', color=colors[idx], lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted_combo[idx], max_score)
ax.text(1.025, 1.1*max_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)
ax.axhline(y=IC_Dem, ls='--', color=colors[idx3], alpha=1, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted_combo[idx3], IC_Dem)
ax.text(1.025, 1.0*IC_Dem, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)
idx = np.argmax(y[idx1:idx2])+idx1
ax.axhline(y=max_score_progscores, ls='--', color=colors[idx], alpha=1, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted_combo[idx], max_score_progscores)
ax.text(1.025, 0.725*max_score_progscores, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

if score=='roc_auc_score':
    name_score = 'auc'

filename = '%sMultiv' % (name_score) + '.pdf'    
saving_str = path_figures + filename
#plt.savefig(saving_str, bbox_inches='tight') # dpi=1000, 

#plt.bar(x_positions_bars, perf)
plt.show()

In [ ]:
magnification = 1.35
ratio = 2./5.

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'roc_auc_score'

Data_to_use = Data_combo

y = Data_to_use.loc[old_labels_sorted_combo, (score, 'Mean')].values
y_lower = Data_to_use.loc[old_labels_sorted_combo, (score, 'CI_lower')].values
y_upper = Data_to_use.loc[old_labels_sorted_combo, (score, 'CI_upper')].values

ax.bar(x_positions_bars, y, color=colors, alpha=0.8, width=width, lw=lines_w)
ax.errorbar(x_positions_bars, y, yerr=[y-y_lower,y_upper-y], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_combo, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('auc')
ax.set_ylim([0.3, .9])
ax.set_yticks([0.3, 0.6, 0.9])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    

idx = np.argmax(y)
ax.axhline(y=max_score, ls='--', color=colors[idx], lw=0.5)
ax.axhline(y=IC_Dem, ls='--', color=colors[idx3], alpha=1, lw=0.5)
idx = np.argmax(y[idx1:idx2])+idx1
ax.axhline(y=max_score_progscores, ls='--', color=colors[idx], alpha=1, lw=0.5)

if score=='roc_auc_score':
    name_score = 'auc'

filename = '%sMultiv_noText' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str, bbox_inches='tight')
plt.show()

#### minNPV - NPV

In [ ]:
Data_combo_0 = pd.concat([Data_univ_0.loc[old_labels_sorted, :], Data_0.loc[multiv_models, :]], axis=0)

In [ ]:
magnification = 1.5
ratio = 2./5.25

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'NPV'
name_score = 'npv'

y = Data_combo_0.loc[old_labels_sorted_combo, (score, 'Mean')].values
y_lower = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_lower')].values
y_upper = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_upper')].values

minNPV_train = min_NPV
delta_minNPV_test = 0.02
mask_npv_high = y_upper>minNPV_train-delta_minNPV_test
x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_low = mask_npv_high==False
x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.3
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_none = (y>0.)==False
x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.06*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_combo, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel(name_score)
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    
    
minNPV_train = min_NPV
ax.axhline(y=minNPV_train-0.02, ls='--', color='black', alpha=1, lw=0.5)
my_string = 'min. val.\naccepted:\n%.2f' % (minNPV_train-0.02)
ax.text(1.025, 1.1*max_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'light: npv$<$%.2f' % (minNPV_train-0.02)
ax.text(0.04, 1.1, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'dark: npv$\ge$%.2f' % (minNPV_train-0.02)
ax.text(0.5, 1.1, my_string,
           horizontalalignment='center',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'x: missing'
ax.text(.96, 1.1, my_string,
           horizontalalignment='right',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

ax.set_ylim([0., 1.])
ax.set_yticks([0., 0.5, 1.])
filename = 'minNPV#%sMultiv' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()

In [ ]:
magnification = 1.35
ratio = 2./5.

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'NPV'
name_score = 'npv'

y = Data_combo_0.loc[old_labels_sorted_combo, (score, 'Mean')].values
y_lower = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_lower')].values
y_upper = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_upper')].values

minNPV_train = min_NPV
delta_minNPV_test = 0.02
mask_npv_high = y_upper>minNPV_train-delta_minNPV_test
x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_low = mask_npv_high==False
x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.3
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_none = (y>0.)==False
x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.06*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_combo, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel(name_score)
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    
    
minNPV_train = min_NPV
ax.axhline(y=minNPV_train-0.02, ls='--', color='black', alpha=1, lw=0.5)
ax.set_ylim([0., 1.])
ax.set_yticks([0., 0.5, 1.])
filename = 'minNPV#%sMultiv_noText' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str,bbox_inches='tight') # dpi=1000
plt.show()

#### minNPV - specificity

In [ ]:
magnification = 1.5
ratio = 2./5.25

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'specificity'
name_score = 'specificity'

y = Data_combo_0.loc[old_labels_sorted_combo, (score, 'Mean')].values
y_lower = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_lower')].values
y_upper = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_upper')].values

x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_low = mask_npv_high==False
x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.3
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_none = (y>0.)==False
x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.04*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_combo, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('f. low risk')
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    

y_l = y
y_l[mask_npv_low] = 0
max_score = max(y_l)
idx = np.argmax(y_l)
ax.axhline(y=max_score, ls='--', color=colors[idx], lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted_combo[idx], max_score)
ax.text(1.025, 1.5*max_score, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)
idx1 = len_dem+len_fc+len_ck+len_bm
idx2 = idx1 + len_scores
max_score_progscores = max(y_l[idx1:idx2])
idx = np.argmax(y_l[idx1:idx2])+idx1
ax.axhline(y=max_score_progscores, ls='--', color=colors[idx], alpha=1, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted_combo[idx], max_score_progscores)
ax.text(1.025, 1.*max_score_progscores, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)
idx1 = len_dem+len_fc+len_ck+len_bm+len_scores
idx2 = idx1 + 2
max_score_FC = max(y_l[idx1:idx2])
idx = np.argmax(y_l[idx1:idx2])+idx1
ax.axhline(y=max_score_FC, ls='--', color=colors[idx], alpha=1, lw=0.5)
my_string = '%s: %.2f' % (x_labels_sorted_combo[idx], max_score_FC)
ax.text(1.025, 1.*max_score_FC, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

ax.set_ylim([0., .8])
ax.set_yticks([0., 0.2, .4, 0.6, 0.8])
filename = 'minNPV#%sMultiv' % (name_score) + '.pdf'    
saving_str = path_figures + filename
#plt.savefig(saving_str, bbox_inches='tight') # dpi=1000
plt.show()

In [ ]:
magnification = 1.35
ratio = 2./5.

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'specificity'
name_score = 'specificity'

y = Data_combo_0.loc[old_labels_sorted_combo, (score, 'Mean')].values
y_lower = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_lower')].values
y_upper = Data_combo_0.loc[old_labels_sorted_combo, (score, 'CI_upper')].values

x_pos_l = x_positions_bars[mask_npv_high]
y_l = y[mask_npv_high]
y_lower_l = y_lower[mask_npv_high]
y_upper_l = y_upper[mask_npv_high]
colors_l = np.array(colors)[mask_npv_high]
alpha = 0.8
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_low = mask_npv_high==False
x_pos_l = x_positions_bars[mask_npv_low]
y_l = y[mask_npv_low]
y_lower_l = y_lower[mask_npv_low]
y_upper_l = y_upper[mask_npv_low]
colors_l = np.array(colors)[mask_npv_low]
alpha = 0.3
ax.bar(x_pos_l, y_l, color=colors_l, alpha=alpha, width=0.6, lw=lines_w)
ax.errorbar(x_pos_l, y_l, yerr=[y_l-y_lower_l,y_upper_l-y_l], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

mask_npv_none = (y>0.)==False
x_pos_l = x_positions_bars[mask_npv_none]
y_l = 0.04*np.ones_like(x_pos_l)
colors_l = np.array(colors)[mask_npv_none]
alpha = 0.8
ax.scatter(x_pos_l, y_l, marker='x', s=15, color=colors_l, alpha=alpha, lw=0.8)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_combo, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('f. low risk')
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    

y_l = y
y_l[mask_npv_low] = 0
max_score = max(y_l)
idx = np.argmax(y_l)
ax.axhline(y=max_score, ls='--', color=colors[idx], lw=0.5)
idx1 = len_dem+len_fc+len_ck+len_bm
idx2 = idx1 + len_scores
max_score_progscores = max(y_l[idx1:idx2])
idx = np.argmax(y_l[idx1:idx2])+idx1
ax.axhline(y=max_score_progscores, ls='--', color=colors[idx], alpha=1, lw=0.5)
idx1 = len_dem+len_fc+len_ck+len_bm+len_scores
idx2 = idx1 + 2
max_score_FC = max(y_l[idx1:idx2])
idx = np.argmax(y_l[idx1:idx2])+idx1
ax.axhline(y=max_score_FC, ls='--', color=colors[idx], alpha=1, lw=0.5)

#ax.set_yscale('log')
ax.set_ylim([0., .8])
ax.set_yticks([0., 0.2, .4, 0.6, 0.8])
filename = 'minNPV#%sMultiv_noText' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()

#### Coefficients (FC+Dem)

In [ ]:
reg_fixed = True
foldername_multiv = 'MultivModels_DOnsetMin#0#Max#30_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NansRowGroupwise#50_PCAPercVarTh#5/'
name_coef = 'FC+Dem'  #Flowcyt All
filename = 'Parameters_model#LR_name#%s.xlsx' % name_coef
path_import = path_results+foldername_multiv
df_LR_coefbias = pd.read_excel(path_import+filename, engine='openpyxl', header=[0], index_col=0)
n_features_model = len(df_LR_coefbias.columns)

In [ ]:
df_LR_coefbias_norm = df_LR_coefbias.values / np.sqrt(np.sum(df_LR_coefbias.values**2, axis=1)).reshape(-1, 1)
df_LR_coefbias_norm = pd.DataFrame(df_LR_coefbias_norm, columns=df_LR_coefbias.columns)

In [ ]:
Data = df_LR_coefbias_norm.copy()

# Labels
old_labels = [col for col in Data.columns]
new_labels = CF.change_names(old_labels)
x_labels = np.array(new_labels) # or list(X_data.columns.values)

# Sorting
sorting_score = 'roc_auc_score'
y = np.quantile(Data.values, 0.5, axis=0)
idx_sorted = []
for set_col in [['bias'], immunecells_set+demographics_set]:
    sub_idx = np.array([i for i in range(len(old_labels)) if old_labels[i] in set_col])
    min_sub_idx = min(sub_idx)
    sub_idx_sorted = min_sub_idx + np.argsort(y[sub_idx])
    idx_sorted.extend(sub_idx_sorted)
idx_sorted = np.array(idx_sorted)
x_labels_sorted = np.array(new_labels)[idx_sorted]
old_labels_sorted =  np.array(old_labels)[idx_sorted]


# Colors
colors = ['black'] + [color_IC_multiv]*(len(x_labels_sorted)-1)

In [ ]:
# Exclude bias
x_labels_sorted_l = x_labels_sorted[1:]
old_labels_sorted_l = old_labels_sorted[1:]
colors_l = list(np.array(colors)[1:])

x_range = np.arange(len(x_labels_sorted_l), dtype=float)
x_positions_labels = x_range

magnification = 0.83
ratio = 2./3.2

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

alpha = 0.6
widths = 0.5
showfliers = False
showcaps = False
alpha_whisk = 1.

ax.axhline(y=0, color=Parameters.grey_1, lw=0.5, alpha=1.)

colors_labels = []
for i, element in enumerate(old_labels_sorted_l):
    color = colors_l[i]
    x = Data[element].values
    colors_labels.append(color)
    
    bp = ax.boxplot(x=x, positions=[x_positions_labels[i]],
                    widths=widths, showfliers=showfliers, showcaps=showcaps, patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    plt.setp(bp['whiskers'], color='black', lw=0.5, alpha=alpha_whisk)
    for box in bp['boxes']:
        box.set(color=color, linewidth=0.7, alpha=1.)
        box.set(facecolor=color, alpha=alpha)
        

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_l, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('LR coef: FC,Dem')
ax.set_ylim([-0.8, 0.8])
ax.set_yticks([-0.6, -0.3, 0., 0.3, 0.6])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors_labels):
    xtick.set_color('black')   
    
filename = 'Coef_Model#FCDem'
filename = filename + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str, bbox_inches='tight') # dpi=1000, 
plt.show()

#### Coefficients (CK+Dem)

In [ ]:
name_coef = 'CK+Dem'  #Flowcyt All
filename = 'Parameters_model#LR_name#%s.xlsx' % name_coef
path_import = path_results+foldername_multiv
df_LR_coefbias = pd.read_excel(path_import+filename, engine='openpyxl', header=[0], index_col=0)
n_features_model = len(df_LR_coefbias.columns)

In [ ]:
# Normalise coefficients
df_LR_coefbias_norm = df_LR_coefbias.values / np.sqrt(np.sum(df_LR_coefbias.values**2, axis=1)).reshape(-1, 1)
df_LR_coefbias_norm = pd.DataFrame(df_LR_coefbias_norm, columns=df_LR_coefbias.columns)

In [ ]:
Data = df_LR_coefbias_norm.copy()

# Labels
old_labels = [col for col in Data.columns]
new_labels = CF.change_names(old_labels)
x_labels = np.array(new_labels) # or list(X_data.columns.values)

# Sorting
sorting_score = 'roc_auc_score'
y = np.quantile(Data.values, 0.5, axis=0)
idx_sorted = []
for set_col in [['bias'], cytokines_set+demographics_set]:
    sub_idx = np.array([i for i in range(len(old_labels)) if old_labels[i] in set_col])
    min_sub_idx = min(sub_idx)
    sub_idx_sorted = min_sub_idx + np.argsort(y[sub_idx])
    idx_sorted.extend(sub_idx_sorted)
idx_sorted = np.array(idx_sorted)
x_labels_sorted = np.array(new_labels)[idx_sorted]
old_labels_sorted =  np.array(old_labels)[idx_sorted]


# Colors
colors = ['black'] + [color_CK_multiv]*(len(x_labels_sorted)-1)

In [ ]:
# Exclude bias
x_labels_sorted_l = x_labels_sorted[1:]
old_labels_sorted_l = old_labels_sorted[1:]
colors_l = list(np.array(colors)[1:])

x_range = np.arange(len(x_labels_sorted_l), dtype=float)
x_positions_labels = x_range

magnification = 0.775
ratio = 2./3.5

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

alpha = 0.6
widths = 0.5
showfliers = False
showcaps = False
alpha_whisk = 1.

ax.axhline(y=0, color=Parameters.grey_1, lw=0.5, alpha=1.)

colors_labels = []
for i, element in enumerate(old_labels_sorted_l):
    color = colors_l[i]
    x = Data[element].values
    colors_labels.append(color)
        
    bp = ax.boxplot(x=x, positions=[x_positions_labels[i]],
                    widths=widths, showfliers=showfliers, showcaps=showcaps, patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    plt.setp(bp['whiskers'], color='black', lw=0.5, alpha=alpha_whisk)
    for box in bp['boxes']:
        box.set(color=color, linewidth=0.7, alpha=1.)
        box.set(facecolor=color, alpha=alpha)
        

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_l, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('LR coef: CK,Dem')
ax.set_ylim([-0.8, 0.8])
ax.set_yticks([-0.6, -0.3, 0., 0.3, 0.6])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors_labels):
    xtick.set_color('black') 
    
filename = 'Coef_Model#CKDem'
filename = filename + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str, bbox_inches='tight') # dpi=1000, 
plt.show()

#### Coefficients (BM+Dem)

In [ ]:
name_coef = 'BM+Dem'
filename = 'Parameters_model#LR_name#%s.xlsx' % name_coef
path_import = path_results+foldername_multiv
df_LR_coefbias = pd.read_excel(path_import+filename, engine='openpyxl', header=[0], index_col=0)
n_features_model = len(df_LR_coefbias.columns)

In [ ]:
# Normalise coefficients
df_LR_coefbias_norm = df_LR_coefbias.values / np.sqrt(np.sum(df_LR_coefbias.values**2, axis=1)).reshape(-1, 1)
df_LR_coefbias_norm = pd.DataFrame(df_LR_coefbias_norm, columns=df_LR_coefbias.columns)

In [ ]:
Data = df_LR_coefbias_norm.copy()

# Labels
old_labels = [col for col in Data.columns]
new_labels = CF.change_names(old_labels)
x_labels = np.array(new_labels) # or list(X_data.columns.values)

# Sorting
sorting_score = 'roc_auc_score'
y = np.quantile(Data.values, 0.5, axis=0)
idx_sorted = []
for set_col in [['bias'], biomarkers_set+demographics_set]:
    sub_idx = np.array([i for i in range(len(old_labels)) if old_labels[i] in set_col])
    min_sub_idx = min(sub_idx)
    sub_idx_sorted = min_sub_idx + np.argsort(y[sub_idx])
    idx_sorted.extend(sub_idx_sorted)
idx_sorted = np.array(idx_sorted)
x_labels_sorted = np.array(new_labels)[idx_sorted]
old_labels_sorted =  np.array(old_labels)[idx_sorted]


# Colors
colors = ['black'] + [color_BM_multiv]*(len(x_labels_sorted)-1)

In [ ]:
# Exclude bias
x_labels_sorted_l = x_labels_sorted[1:]
old_labels_sorted_l = old_labels_sorted[1:]
colors_l = list(np.array(colors)[1:])

x_range = np.arange(len(x_labels_sorted_l), dtype=float)
x_positions_labels = x_range

magnification = 0.66
ratio = 2./2.7

CF.SetPlotParams(magnification=magnification, ratio=ratio, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

alpha = 0.6
widths = 0.5
showfliers = False
showcaps = False
alpha_whisk = 1.

ax.axhline(y=0, color=Parameters.grey_1, lw=0.5, alpha=1.)

colors_labels = []
for i, element in enumerate(old_labels_sorted_l):
    color = colors_l[i]
    x = Data[element].values
    colors_labels.append(color)
        
    bp = ax.boxplot(x=x, positions=[x_positions_labels[i]],
                    widths=widths, showfliers=showfliers, showcaps=showcaps, patch_artist=True)
    for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
        plt.setp(bp[element], color='black', lw=0.5, alpha=1.)      
    plt.setp(bp['whiskers'], color='black', lw=0.5, alpha=alpha_whisk)
    for box in bp['boxes']:
        box.set(color=color, linewidth=0.7, alpha=1.)
        box.set(facecolor=color, alpha=alpha)
        

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(x_labels_sorted_l, rotation=90)
ax.set_xlim([min(x_positions_labels)-1, max(x_positions_labels)+1])
ax.set_ylabel('LR coef: BM,Dem')
ax.set_ylim([-0.8, 0.8])
ax.set_yticks([-0.6, -0.3, 0., 0.3, 0.6])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors_labels):
    xtick.set_color('black')   
    
filename = 'Coef_Model#BMDem'
filename = filename + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str, bbox_inches='tight') # dpi=1000, 
plt.show()

# Multivariate models - age stratification

In [ ]:
# Parameters
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
capsize = 0
width = 0.35

# Models names
multiv_models = ['FC',
                 'FC+Dem',
                 'CK', 
                 'CK+Dem',
                 'BM', 
                 'BM+Dem']
v = [element.replace('LR: ', '').replace('+', ',').replace('.', '') for element in multiv_models]
multiv_models_names = v

### Import files

In [ ]:
foldername_multiv_1 = 'MultivModels_DOnsetMin#0#Max#30_AgeMin#30#Max#70_Target#OTI+death_Std#PowerTransformer_NansRowGroupwise#50_PCAPercVarTh#5/'
foldername_multiv_2 = 'MultivModels_DOnsetMin#0#Max#30_AgeMin#70#Max#100_Target#OTI+death_Std#PowerTransformer_NansRowGroupwise#50_PCAPercVarTh#5/'

In [ ]:
path_import = path_results+foldername_multiv_1
filename = 'performances_test.xlsx'
df_performances_1 = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)
#
path_import = path_results+foldername_multiv_2
filename = 'performances_test.xlsx'
df_performances_2 = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)

In [ ]:
# Clean nans
df_performances_1 = df_performances_1.where(df_performances_1.notnull().values, 0)
df_performances_2 = df_performances_2.where(df_performances_2.notnull().values, 0)

### Scores

In [ ]:
Data_1 = df_performances_1
Data_2 = df_performances_2
scores = [element[0] for element in Data_1.columns]
scores = list(dict.fromkeys(scores))

index = [col.replace('LR: ', '') for col in Data_1.index]
Data_1.index = index
index = [col.replace('LR: ', '') for col in Data_2.index]
Data_2.index = index

### Plot

#### AUC

In [ ]:
x_range = np.arange(len(multiv_models), dtype=float)
x_positions_labels = x_range
x_positions_bars = x_positions_labels

colors = [color_IC_multiv]*2 + [color_CK_multiv]*2 + [color_BM_multiv]*2

magnification = .68
ratio = 2./2.225

fig_width = 2.9 * magnification # width in inches
fig_height = 2.9 * magnification * ratio  # height in inches
fig_width = 3.0 * magnification # width in inches

CF.SetPlotParams(height=fig_height, width=fig_width, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'roc_auc_score'

y_1 = Data_1.loc[multiv_models, (score, 'Mean')].values
y_lower_1 = Data_1.loc[multiv_models, (score, 'CI_lower')].values
y_upper_1 = Data_1.loc[multiv_models, (score, 'CI_upper')].values
y_2 = Data_2.loc[multiv_models, (score, 'Mean')].values
y_lower_2 = Data_2.loc[multiv_models, (score, 'CI_lower')].values
y_upper_2 = Data_2.loc[multiv_models, (score, 'CI_upper')].values

ax.bar(x_positions_bars-0.18, y_1, color=colors, alpha=0.9, width=width, lw=lines_w)
ax.errorbar(x_positions_bars-0.18, y_1, yerr=[y_1-y_lower_1,y_upper_1-y_1], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)
ax.bar(x_positions_bars+0.18, y_2, color=colors, alpha=0.5, width=width, lw=lines_w)
ax.errorbar(x_positions_bars+0.18, y_2, yerr=[y_2-y_lower_2,y_upper_2-y_2], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(multiv_models_names, rotation=90)
ax.set_xlim([min(x_positions_labels)-0.7, max(x_positions_labels)+0.7])
ax.set_ylabel('auc')
ax.set_ylim([0.5, .9])
ax.set_yticks([0.5, 0.6, 0.7, 0.8, 0.9])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    
    
max_score = max(y_1)
idx = np.argmax(y_1)
ax.axhline(y=max_score, ls='--', dashes=(3., 3.), color=colors[idx], alpha=1., lw=0.5)
my_string = 'best: %.2f' % (max_score)
print(my_string)

my_string = 'dark: age$<$70'
ax.text(-0., 1.2, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'light: age$\ge$70'
ax.text(1., 1.2, my_string,
           horizontalalignment='right',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

if score=='roc_auc_score':
    name_score = 'auc'

filename = '%sMultiv#AgeStratification' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()

# Multivariate models - delta_onset stratification

In [ ]:
# Parameters
fontsize = 9
ylabelsize = 7
xlabelsize = 7 
tex = True
axes_lines_w = 0.5
lines_w = 0.5
capsize = 0
width = 0.35

# Models names
multiv_models = ['FC',
                 'FC+Dem',
                 'CK', 
                 'CK+Dem',
                 'BM', 
                 'BM+Dem']
v = [element.replace('LR: ', '').replace('+', ',').replace('.', '') for element in multiv_models]
multiv_models_names = v

### Import files

In [ ]:
foldername_multiv_1 = 'MultivModels_DOnsetMin#0#Max#10_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NansRowGroupwise#50_PCAPercVarTh#5/'
foldername_multiv_2 = 'MultivModels_DOnsetMin#11#Max#30_AgeMin#30#Max#100_Target#OTI+death_Std#PowerTransformer_NansRowGroupwise#50_PCAPercVarTh#5/'


In [ ]:
path_import = path_results+foldername_multiv_1
filename = 'performances_test.xlsx'
df_performances_1 = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)
#
path_import = path_results+foldername_multiv_2
filename = 'performances_test.xlsx'
df_performances_2 = pd.read_excel(path_import+filename, engine='openpyxl', header=[0, 1], index_col=0)

In [ ]:
# Clean nans
df_performances_1 = df_performances_1.where(df_performances_1.notnull().values, 0)
df_performances_2 = df_performances_2.where(df_performances_2.notnull().values, 0)

### Scores

In [ ]:
Data_1 = df_performances_1
Data_2 = df_performances_2
scores = [element[0] for element in Data_1.columns]
scores = list(dict.fromkeys(scores))

index = [col.replace('LR: ', '') for col in Data_1.index]
Data_1.index = index
index = [col.replace('LR: ', '') for col in Data_2.index]
Data_2.index = index

### Plot

#### AUC

In [ ]:
x_range = np.arange(len(multiv_models), dtype=float)
x_positions_labels = x_range
x_positions_bars = x_positions_labels

colors = [color_IC_multiv]*2 + [color_CK_multiv]*2 + [color_BM_multiv]*2

magnification = .68
ratio = 2./2.225

fig_width = 2.9 * magnification # width in inches
fig_height = fig_width*ratio  # height in inches
fig_width = 3.0 * magnification # width in inches

CF.SetPlotParams(height=fig_height, width=fig_width, fontsize=fontsize, ylabelsize=ylabelsize,
                 xlabelsize=xlabelsize, tex=tex, axes_lines_w=axes_lines_w, lines_w=lines_w, legendmarker=True)

fig, ax = plt.subplots()

score = 'roc_auc_score'

y_1 = Data_1.loc[multiv_models, (score, 'Mean')].values
y_lower_1 = Data_1.loc[multiv_models, (score, 'CI_lower')].values
y_upper_1 = Data_1.loc[multiv_models, (score, 'CI_upper')].values
y_2 = Data_2.loc[multiv_models, (score, 'Mean')].values
y_lower_2 = Data_2.loc[multiv_models, (score, 'CI_lower')].values
y_upper_2 = Data_2.loc[multiv_models, (score, 'CI_upper')].values

ax.bar(x_positions_bars-0.18, y_1, color=colors, alpha=0.9, width=0.35, lw=lines_w)
ax.errorbar(x_positions_bars-0.18, y_1, yerr=[y_1-y_lower_1,y_upper_1-y_1], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)
ax.bar(x_positions_bars+0.18, y_2, color=colors, alpha=0.5, width=0.35, lw=lines_w)
ax.errorbar(x_positions_bars+0.18, y_2, yerr=[y_2-y_lower_2,y_upper_2-y_2], linestyle='none', ecolor='black', fmt="-", lw=lines_w, capsize=capsize)

ax.set_xticks(x_positions_labels)
ax.set_xticklabels(multiv_models_names, rotation=90)
ax.set_xlim([min(x_positions_labels)-0.7, max(x_positions_labels)+0.7])
ax.set_ylabel('auc')
ax.set_ylim([0.5, .9])
ax.set_yticks([0.5, 0.6, 0.7, 0.8, 0.9])
ax.xaxis.set_ticks_position('none') 
ax.tick_params(axis='x', which='major', pad=2)

for xtick, color in zip(ax.get_xticklabels(), colors):
    xtick.set_color(color)    
    
max_score = max(y_2)
idx = np.argmax(y_2)
ax.axhline(y=max_score, ls='--', dashes=(3., 3.), color=colors[idx], alpha=1., lw=0.5)
my_string = 'best: %.2f' % (max_score)
print(my_string)

my_string = 'dark: $\mathrm{\Delta t_{ons}}\le$10'
ax.text(-.1, 1.2, my_string,
           horizontalalignment='left',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

my_string = 'light: $\mathrm{\Delta t_{ons}}>$10'
ax.text(1.0, 1.2, my_string,
           horizontalalignment='right',
           verticalalignment='center',
           rotation='horizontal',
           transform=ax.transAxes,
           fontsize=ylabelsize)

if score=='roc_auc_score':
    name_score = 'auc'

filename = '%sMultiv#DonsetStratification' % (name_score) + '.pdf'    
saving_str = path_figures + filename
plt.savefig(saving_str,bbox_inches='tight') # dpi=1000, 
plt.show()